##### **load pickle data**

In [ ]:
# read pickle data
import pickle

trainX = pickle.loads(open("C:/Users/AbsSayem/.vscode/computer_vision/dataset/HARDataset1/trainX.pickle", "rb").read())
trainY = pickle.loads(open("C:/Users/AbsSayem/.vscode/computer_vision/dataset/HARDataset1/trainY.pickle", "rb").read())
testX  = pickle.loads(open("C:/Users/AbsSayem/.vscode/computer_vision/dataset/HARDataset1/testX.pickle", "rb").read())
testY  = pickle.loads(open("C:/Users/AbsSayem/.vscode/computer_vision/dataset/HARDataset1/testY.pickle", "rb").read())

In [ ]:
CATEGORIES = ["Boxing", "HandClapping", "HandWaving", "Jogging", "Running", "Standing", "Walking"]
print(CATEGORIES[trainY[0]])

**show images**

In [ ]:
# read images from pickle data
import matplotlib.pyplot as plt
# show a single image
#plt.imshow(trainX[0], cmap='gray')

# show multiple images
CATEGORIES = ["Boxing", "HandClapping", "HandWaving", "Jogging", "Running", "Standing", "Walking"]
num_images = 10
# grid for subplots
rows, cols = 2, 5
# make a grid
fig, axes = plt.subplots(rows, cols, figsize=(13,4))
# Flatten the axes array for easy iteration
axes = axes.flatten()

# read and place images in the plot
for i in range(min(num_images, rows * cols)):
    axes[i].imshow(trainX[i], cmap='gray')  # Use 'cmap=None' for color images
    axes[i].set_title(f'{CATEGORIES[trainY[i]]}')
    #axes[i].axis('off')  # Optional: Turn off axis labels
# Adjust layout for better spacing
plt.tight_layout()
plt.show()

In [ ]:
# access the frames
for iter in range(len(trainX)):    # len(trainX)=45952
    # when iter==0: it gets the array of image 0
    pass    

`here we are able to access the frames. we will return back here after extracting foreground(human_shape) and getting the gait data`

**foreground extraction**
* to segment and seperate foreground we will use OpenCV and GrabCut.
* how the GrabCut algorithm works?- by repetating the following steps-
    1. Estimate the color distribution of the foreground and background via a Gaussian Mixture Model (GMM)
    2. Construct a Markov random field over the pixels labels (i.e., foreground vs. background)
    3. Apply a graph cut optimization to arrive at the final segmentation
* OpenCV has an implementation of GrabCut via the `cv2.grabCut` function

**opencv grabcut: foreground segmentation and extraction - let's learn about opencv grabcut function:**
* `grabCut(img, mask, rect, bgdModel, fgdModel, iterCount[, model]) -> mask, bgdModel, fgdModel`
* `input arguments`:
    * `img`: the input image, (unsigned 8-bit integer in BGR (3) channel ordering)
    * `mask`: the input/output mask (a single-channel unsigned 8-bit integer).
        * `cv2.GC_INIT_WITH_RECT`: the bounding box initialization, mask is initiated automatically
        * `cv2.GC_INIT_WITH_MASK`: the mask initialization
    * `rect`: the bounding box rectangle that contains the region that we want to segment, only used when we set the `mode` to `cv2.GC_INIT_WITH_MASK`
    * `bgModel`: temporary array used by GrabCut internally when modeling the background (no need external manipulation)
    * `fgModel`: temporary array used by GrabCut internally when modeling the foreground (no need external manipulation)
    * `mode`: either `cv2.GC_INIT_WITH_RECT` or `cv2.GC_INIT_WITH_MASK`, depends on what I am initializing
* `output arguments`:
    * `mask`: the output mask after applying GrabCut
    * `bgModel`: the temporary array used to model background (ignorable)
    * `fgModel`: the temporary array used to model foreground (ignorable)

**implement grabcut initializer (bounding_box vs mask)**

`bounding box`
* specify the bounding box is then used by the grabcut function to segment the foreground
* bounding box can be generated by -
    * manually labelling the (x,y)-coordinates of the bounding box
    * applying a `Haar Cascade` algorithm
    * using `HOG` + `Linear SVM` to detect the object
    * using dl-based object detector, like- `Faster R-CNN`, `SSDs`, `YOLO` etc
* `we will use haar cascade algorithm, because - it is suitable for single activity, also very fast.`

In [ ]:
# bounding box using haar casecade
def bounding_box(img):
    import matplotlib.pyplot as plt
    import cv2
    harcascade = "models/haarcascade_fullbody.xml"
    fullcascade = cv2.CascadeClassifier(harcasecade)
    #img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    full_body = fullcascade.detectMultiScale(img, 1.1, 4)
    for (x,y,w,h) in full_body:
        cv2.ractangle(img, (x,y), (x+w, y+h), (0,0,255), 2)
    plt.imshow(img)

In [1]:
import cv2
import matplotlib.pyplot as plt
harcascade = "models/haarcascade_frontalface_default.xml"

cap = cv2.VideoCapture(0)
cap.set(3,640)  # width
cap.set(4,480)  # height

while True:
    success, img = cap.read()
    facecascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = facecascade.detectMultiScale(img_gray, 1.1,4)
    for (x,y,w,h) in face:
        cv2.rectangle(img, (x,y), (x+w, h+h), (0,0,255), 2)
    #cv2.imshow("Face", img)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
# check using a frame
import matplotlib.pyplot as plt
import cv2
img1 = trainX[0]
print(img1.shape)
#plt.imshow(img1)
#bounding_box(img1)

`masks`
* using mask, we can supply the approximate segmentation of the object in the image. grabcut can then iteratively apply graph cuts to improve the segmentation and extract the foreground from the image.
* the masks could be generated by -
    * manually creating in photo editing software, like- `photoshop`
    * applying ip operations, like- `thresholding`, `edge-detection`, `contour filtering`
    * utilizing dl-based segmentation nets, like- `Mask R-CNN`, `U-NET`

In [ ]:
# masks 
def masks():
    pass